# Object Detection and Fusion in a Camera Network
The goal of the project is to combine multiple measurements from a distributed
sensor network, gathered centrally and fused to produce a location estimate with
less uncertainty than one obtainable from only a single sensor. Five raspberry pi powered cameras mounted on a frame aimed at the ground are employed to track an object which is moving in a circular path at a constant
angular velocity at regular time intervals.

These images are used to detect the position of the object using an object
detection algorithm and are used together in Kalman Filter to
estimate the location with decreased uncertainity.

## Object Detection

The uncertain measurements are obtained using off the shelf harris corner detection object detection algorithm. We used a mask of a very narrow color space to make the red-colored one on the top of our object(a car) prominent. The harris corner detection gives us possible corners which are aggregated to get an average uncertain measurement. For details of the algorithm feel free to have look at  [detector.py](files/notebook2.ipynb)





<br>
![](misc/detection.png)

## Raspberry-Pi Inter Communication
Communication among the pi modules is
achieved by building a server-client architecture with a
local machine as a central server node and all other pi’s
as connecting client nodes. Both server and the clients
should be in the same network to be able to
communicate with each other. The communication is achieved using the Python Socketio library. 



![](misc/pi-inter.PNG)<br>

Our goal is to develop an architecture where upon connection of all 5 clients the server activates all the cameras to take synchronous photos after regular intervals Details are given in [client.py](files/notebook2.ipynb) and [server.py](files/notebook2.ipynb)



```
libcamera-still -t {args.t} --width 1280 --height 960 --timelapse {args.timelapse} client_{args.client}_%03d.jpg'
```
<br>
![](misc/photos-interval.png)

## Camera Callibration
After obtaining the readings from alll the camera the next task was to make the readings comparable. Since cameras had different orientations and were displaced at known distances from each other we applied translational and rotational transformations to get all readings in the same frame of reference.

![](misc/callibration-2.png)
<br>
Translational distance are estimated using a chess board to measure the pixel to distance ratio. This ratio is then used to find the translations in pixel units as we have the real translations available. For reference look at [get_coordinates.py](files/notebook2.ipynb)

<br>
![](misc/callibration-1.png)
$$\begin{bmatrix}
    X_{global}  \\
    Y_{global} \\
\end{bmatrix}= \begin{bmatrix}
    X_t  \\
    Y_t \\
\end{bmatrix} + \begin{bmatrix}
    \cosθ & -\sinθ \\
    \sinθ & \cosθ \\
\end{bmatrix} . \begin{bmatrix}
    x_{local}  \\
    y_{local} \\
\end{bmatrix}
$$

## Kalman Filter
This is the implementation of constant turn model. Consider a situation where our object under observation is moving in a circular path at constant velocity. The positions of the object are measured at each timestamp through the camera sensors. Lets begin with defining the state space vector of our object 

###State Vector
The state space vector can be defined as:
$${x_k}= \begin{bmatrix}
    x_{k}  \\
    \dot x_k \\
    y_{k} \\
    \dot y_k \\
\end{bmatrix}
$$

$$
{x_{k+1}} = \textbf{A}.{x_k} + \textbf{B}{u_k} + {w_k}
$$
As the control input = 0
$$\begin{bmatrix}
    x_{k+1}  \\
    \dot x_{k+1} \\
    y_{k+1} \\
    \dot y_{k+1} \\
\end{bmatrix}= \textbf{A}.\begin{bmatrix}
    x_{k}  \\
    \dot x_k \\
    y_{k} \\
    \dot y_k \\
\end{bmatrix}
$$


$$\begin{bmatrix}
    x_{k+1}  \\
    \dot x_{k+1} \\
    y_{k+1} \\
    \dot y_{k+1} \\
\end{bmatrix}= \begin{bmatrix}
    1&\frac{\sinωT}{ω}&0&-\frac{1-\cosωT}{ω}  \\
    0&\cosωT&0&-\sinωT\\
    0&\frac{1-\cosωT}{ω}&1&\frac{\sinωT}{ω} \\
    0&-\sinωT&0&\cosωT \\
\end{bmatrix}.\begin{bmatrix}
    x_{k}  \\
    \dot x_k \\
    y_{k} \\
    \dot y_k \\
\end{bmatrix}
$$
<br>
Here ω is the constant angular velocity measured through taking an average of the turn taken by the object in each intervals. This can be done using photos at regular intervals




### Observation Model

$$
z_k = \textbf{H}.x_k +v_k 
$$

$$
z_k = \begin{bmatrix}
    1&0&0&0  \\
    0&0&1&0 \\
\end{bmatrix}. x_k
$$ 

### Error Covariance
$$ w_k \sim \mathcal{N}(0,\,\textbf{Q})$$

$$\textbf{Q} = S \begin{bmatrix}
    σ_x^2&0&σ_{x \Delta x}&0  \\
    0&σ_y^2&0&σ_{y \Delta y}\\
    σ_{\Delta x x}&0&σ_{\Delta x}^2&0 \\
    0&σ_{\Delta y y}&0&σ_{\Delta y}^2 \\
\end{bmatrix}$$

$$\textbf{Q} = S \begin{bmatrix}
    \frac{T^3}{3}&\frac{T^2}{2}&0&0  \\
    \frac{T^2}{2}&T&0&0\\
    0&0&\frac{T^3}{3}&\frac{T^2}{2} \\
    0&0&\frac{T^2}{2}&T\\
\end{bmatrix}$$




### Measurement Covariance
$$ v_k \sim \mathcal{N}(0,\,\textbf{R})$$
$$ \textbf{R} = σ^2 \textbf{I}$$
$$\textbf{R} =  \begin{bmatrix}
    σ_x^2&0  \\
    0&σ_y^2\\
\end{bmatrix}$$

### Initial Uncertainity 
$$\textbf{P} = \begin{bmatrix}
    σ_x^2&0&0&0  \\
    0&σ_{Δx}^2&0&0 \\
    0&0&σ_x^2&0 \\
    0&0&0&σ_{Δy}^2 \\
\end{bmatrix} $$
 




### Prediction Step
$${x_{k+1}} = A.{x_k} 
$$

$$
P_{k+1} = A_k.P_k.A^T +Q
$$

### Update Step

For each measurement from camera:

Computing the Kalman Gain:
$$
K_k = P_k.H^T(H.P_k.H^T+R)^{-1} 
$$

Updating the estimate using position measurements from the camera:
$$
x_k = x_k+K_k(z_x-H.x_k)
$$

Updating the error covariance matrix:
$$
P_k = (I-K_k.H)P_k
$$

For reference look at [filter.py](files/notebook2.ipynb)